In [1]:
import regex as re

In [2]:
# text = """Docker là nền tảng cung cấp cho các công cụ, service để các developers, adminsystems có thể phát triển, thực thi, chạy các ứng dụng với containers. Hay nói một cách khác nó là một nền tảng để cung cấp cách để building, deploy và run các ứng dụng một cách dễ dàng trên nền tảng ảo hóa - "Build once, run anywhere". Hay nói một cách dễ hiểu như sau: Khi chúng ta muốn chạy app thì chúng ta phải thiết lập môi trường chạy cho nó. Thay vì chúng ta sẽ đi cài môi trường chạy cho nó thì chúng ta sẽ chạy docker.

# Ứng dụng Docker chạy trong vùng chứa (container) có thể được sử dụng trên bất kỳ hệ thống nào: máy tính xách tay của nhà phát triển, hệ thống trên cơ sở hoặc trong hệ thống đám mây. Và là một công cụ tạo môi trường được "đóng gói" (còn gọi là Container) trên máy tính mà không làm tác động tới môi trường hiện tại của máy, môi trường trong Docker sẽ chạy độc lập.

# Docker có thể làm việc trên nhiều nền tảng như Linux, Microsoft Windows và Apple OS X.
# Virtualization host là gì ?

# Khi chúng ta nói về Virtualization, nó đề cập đến việc nhập hệ điều hành Guest trên hệ điều hành máy chủ, cho phép các nhà phát triển chạy nhiều HĐH trên các máy ảo khác nhau trong khi tất cả chúng chạy trên cùng một máy chủ, do đó loại bỏ nhu cầu cung cấp thêm tài nguyên phần cứng.

# Ưu điểm :

# Kích hoạt nhiều hệ điều hành trên cùng một máy.
# Nó rẻ hơn so với các phương pháp trước đây, do thiết lập cơ sở hạ tầng ít hơn / nhỏ gọn.
# Nếu có bất kỳ trạng thái thất bại nào, thật dễ dàng để phục hồi và bảo trì.
# Cung cấp nhanh hơn các ứng dụng và tài nguyên cần thiết cho các nhiệm vụ.
# Tăng năng suất, hiệu quả và đáp ứng CNTT.
# Virtualization host là gì ?
# Từ kiến trúc VM ở trên, chúng ta có thể hình dung ra rằng 3 hệ điều hành Guest hoạt động như các máy ảo đang chạy trên một hệ điều hành máy chủ. Trong Virtualization, quá trình cấu hình lại phần cứng, phần sụn thủ công, cài đặt HĐH mới, cài đặt hệ điều hành mới có thể hoàn toàn tự động, tất cả các bước này được lưu trữ dưới dạng dữ liệu trong bất kỳ tệp nào của đĩa.

# Trong Virtualization, mỗi ứng dụng và hệ điều hành sống trong một thùng chứa phần mềm riêng biệt có tên Virtural Machine (VM) , trong đó VM hoàn toàn tách biệt, tất cả các tài nguyên điện toán như CPU, lưu trữ và kết nối mạng được gộp chung với nhau. Virtural Machine (VM) về bản chất là một giả lập của một máy tính để thực thi các ứng dụng giống như một máy tính thật. VMs chạy trên một máy vật lý sử dụng một thứ gọi là “hypervisor”. Hypervisor có thể là phần cứng, phần mềm hoặc là một bản firmware nào đó có thể chạy trực tiếp trên máy thật (host machine) có chức năng cho nhiều máy ảo chạy trên nó. Host machine sẽ cung cấp cho VMs những tài nguyên như là RAM, CPU. Những tài nguyên đó sẽ được phân bổ giữa các VMs theo cách mà bạn cho là hợp lý. Nếu một VM chạy nhiều ứng dụng hoặc nặng thì bạn phải cung cấp tài nguyên cho nó nhiều tài nguyên hơn những VMs khác trên cùng một host machine.

# """

In [3]:
text = """xin chào"""

In [4]:
GPT_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [5]:
pattern = re.compile(GPT_SPLIT_PATTERN)

In [6]:
pattern.findall(text)

['xin', ' chào']

In [7]:
text_encode = list(text.encode('utf-8'))

In [8]:
text_encode = list(map(int,text_encode))

In [9]:
def get_stats(ids,pairs:dict|None=None):
    if pairs == None:
        pairs = {}
    for pair in zip(ids[:-1],ids[1:]):
        pairs[pair] = pairs.get(pair,0) + 1
    return pairs




In [10]:
# def best_pair(pairs:dict):
#     return max(pairs,key=pairs.get)

In [11]:
def merge_func(ids:list,best_pair:dict,idx:int):
    '''
    merge a best pair into a new token in the given ids.
    
    args:
        ids: a list of token ids
        best_pair: a tuple of two best pair
        idx: a new idx for the best_pair
    
    return:
        a new list of token ids with best_pair replaced by idx
    '''
    i = 0
    new_ids = [] 

    while i<len(ids):
        if i<len(ids)-1 and ids[i] == best_pair[0] and ids[i+1]==best_pair[1]:
            new_ids.append(idx)
            i+=2
        else:
            new_ids.append(ids[i])
            i+=1
    
    return new_ids

In [12]:
def merge(ids:list,best_pair:tuple,idx:int):
    i = 0
    new_token = []
    while i<len(ids):
        if i<len(ids)-1 and ids[i] == best_pair[0] and ids[i+1] == best_pair[1]:
            new_token.append(idx)
            i+=2
        else:
            new_token.append(ids[i])
            i+=1
    return new_token



# Training

In [13]:
chunks= re.findall(pattern,text)

In [14]:
vocab = {i:bytes([i]) for i in range(256)}
merges ={}
def train(text,vocabsize):
    assert vocabsize >=256, "Your vocabsize must be larger than 256"
    chunks= re.findall(pattern,text)
    encoded_chunks = [list(ch.encode("UTF-8")) for ch in chunks]
    for i in range(vocabsize-256):
        pairs = {}
        for c in encoded_chunks:
            pairs = get_stats(c,pairs)

        if not pairs:
            break

        best_pair = max(pairs,key=pairs.get)
        idx = 256+i
        encoded_chunks = [merge(ids,best_pair,idx) for ids in encoded_chunks]
        merges[best_pair] = idx
        vocab[idx] = vocab[best_pair[0]] + vocab[best_pair[1]]




In [15]:
train(text,500)

In [16]:
merges

{(120, 105): 256,
 (256, 110): 257,
 (32, 99): 258,
 (258, 104): 259,
 (259, 195): 260,
 (260, 160): 261,
 (261, 111): 262}

In [17]:
GPT_SPECIAL_TOKENS = {
    '<|endoftext|>': 100257,
}

In [18]:
SPECIAL_TOKEN_IDX = {
    100257: '<|endoftext|>',
}

# Decode

In [19]:
def decode(ids:list[int]):
    decode_list  = b"".join([SPECIAL_TOKEN_IDX[i].encode("UTF-8") if i in SPECIAL_TOKEN_IDX else vocab[i] for i in ids])
    return decode_list.decode("UTF-8",errors="replace")

# Encode

In [20]:
for i in GPT_SPECIAL_TOKENS:
    print(re.escape(i))

<\|endoftext\|>


In [21]:
def encode_chunks(chunk:list[int])->list[int]:
    if len(chunk)<2:
        return chunk

    while True:
        pair_chunk = get_stats(chunk)
        if not pair_chunk:
            break
        min_pair = min(pair_chunk, key = lambda x: merges.get(x,float("inf")))
        idx = merges.get(min_pair,None)
        if idx == None:
            break 
        chunk = merge_func(chunk,min_pair,idx)
    return chunk





In [22]:
def encode_text(text:str)->list[int]:
    chunks = re.findall(pattern,text)
    ids = []
    for c in chunks:
        encoded_chunk = list(map(int,list(c.encode("UTF-8"))))
        ids.extend(encode_chunks(encoded_chunk))
    return ids
        

In [23]:
def encode(text,special_token:dict|None=None):
    special_patern = None
    if special_token:
        special_patern = "("+"|".join([re.escape(i) for i in special_token]) + ")"
        special_patern  = re.compile(special_patern)
        text = re.split(special_patern,text)
    ids = []
    for t in text:
        if t in special_token:
            ids.append(special_token[t])
        else:
            ids.extend(encode_text(t))
    return ids
    
    

In [24]:
merges

{(120, 105): 256,
 (256, 110): 257,
 (32, 99): 258,
 (258, 104): 259,
 (259, 195): 260,
 (260, 160): 261,
 (261, 111): 262}

In [25]:
text =  "xin chào <|endoftext|>"

In [26]:
encode_text = encode(text,GPT_SPECIAL_TOKENS)

In [ ]:
vocab[].decode("utf-8",errors="replace")

KeyError: 100257

In [27]:
encode_text

[257, 262, 32, 100257]

In [ ]:
decode_text = decode(encode_text)

In [ ]:
decode_text